In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import networkx as nx
import json

In [2]:
cyto.load_extra_layouts()

# Muhaddithat App

This notebook contains all application code for the Muhaddithat dashboard with documentation in markdown. We also have the same code without Jupyter formatting in the file `app.py`, which can be used for deployment on a server.

To start the app, click 'run all cells' and visit http://127.0.0.1:8050/ in your web browser.
Alternatively, you can open a terminal window, navigate to the project directory, and run `python app.py` *(note: `app.y` file has not yet been updated)*. If you use the latter method, make sure you are within a Python environment with all requisite packages installed.

In [3]:
app = JupyterDash(__name__)

## Data

In [4]:
filename = 'data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-1_SEEDS-53.cyjs'

with open(filename, 'r') as f:
    data_53 = json.load(f)

In [5]:
#filename = 'data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-3_SEEDS-53-54-56-59-69-70-71-84-244-2802-10526-10737-11039-11457.cyjs'
filename = 'data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-1_SEEDS-11457.cyjs'

with open(filename, 'r') as f:
    data_11457 = json.load(f)

In [6]:
data = data_53 # set default/initial data being displayed to Aishah bint Abi Bakr's
currentNarrator = 53

In [7]:
with open('data/bios.json', encoding='utf8') as f:
    bios = json.load(f)
    
with open('data/hadith.json', encoding='utf8') as f:
    hadiths = json.load(f)

In [8]:
# testing
# hadiths[0]['NarrationChain']
# print(bios[0]['Biography'])

## Styling

In [9]:
default_stylesheet = [
    {
        'selector': '[gender="m"]',
        'style': {
            'background-color': '#7b7b7b',
            'background-opacity': '0.5',
            'width':'0.01px',
            'height':'0.01px',
            'padding':'3px'
        }
    },
    {
        'selector':'[gender="f"]',
        'style':{
            'background-color':'#ff0000',
            'width':'0.2px',
            'height':'0.2px',
            'padding':'3px'
        }        
    },
    {
        'selector':'[id="1"]',
        'style':{
            'background-color':'#8fce00',
            'width':'5px',
            'height':'5px',
            'background-opacity':'1'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    }    
]

## Layout

Resources on tabs:
- https://dash.plotly.com/dash-core-components#tabs
- https://dash.plotly.com/dash-core-components/tabs


In [10]:
app.layout = html.Div([
    html.Div([
    dcc.Dropdown(
        id='dropdown1',
        options=[{'label': 'Aishah bint Abi Bakr', 'value': '53'} ,
                 {'label': 'Umm al-Darda as-Sughra','value': '11457'}],
        value='53',    
        ),],
        style={'width': '20%', 'display': 'inline-block'}
    ),
    html.Div([
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Bio', value='bio'),
        dcc.Tab(label='Hadiths', value='hadiths'),
    ]),
    html.Div(id='tabs-content'),
    ],
        style={'width':'20%'}
    ),
    
    # section for displaying the graph 
    cyto.Cytoscape(
        id='cytoscape',
        autoungrabify=True, # Prevent users from moving nodes around.
        elements= data['elements'], #edges + nodes,
        layout={
            'name': 'breadthfirst',
            'directed': 'true',
            'maximal': 'true',
            'roots': '1', 
            'circle':'true'
        },
        # style={'width': '100%', 'height': '500px'},
        style={'width':'75%', 'height':'800px'},
        stylesheet=default_stylesheet
    ),
   # dcc.Tooltip(id="graph-tooltip-2", direction='bottom'), # testing
    # testing:
    html.P("Narrator Info:"), # figure out how to put this on the side.
    html.P(id='cytoscape-tapNodeData-output')


])

## Callbacks

https://dash.plotly.com/cytoscape/events


https://dash.plotly.com/cytoscape/callbacks


https://github.com/27shraddhaS/dropdown-graphs-dash/blob/main/dash-dropdown-graph.ipynb


In [11]:
@app.callback(Output('cytoscape-tapNodeData-output', 'children'),
              Input('cytoscape', 'tapNodeData'))
def displayTapNodeData(data):
    if data:
        return "You recently clicked on the node for the narrator: " + data['name']
  
    
# change the graph to view the narrator selected in the dropdown menu

@app.callback(Output('cytoscape', 'elements'),
              Input('dropdown1', 'value'))

def update_elements(value): 
    if value == '11457':
        data = data_11457 # update the data being displayed to the current narrator's
        currentNarrator = '11457' # update the current narrator ID
    if value == '53':
        data = data_53
        currentNarrator = '53'
    return data['elements']
    
# Update the stylesheet so that the edges of the selected narrator are colored 

@app.callback(Output('cytoscape', 'stylesheet'),
              Input('dropdown1', 'value'))

def update_stylesheet(value):
    new_styles = []
    if value == '11457':
        new_styles = [
             {
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    },  
            {
          'selector':  '[source="11457"]',
          'style':{
          'line-color':'#ff0000',
          'width':'0.1px',
          'opacity':'1',
          'target-arrow-color':'#ff0000'} 
        }]
    if value == '53':
        new_styles = [
             {
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    },  {
          'selector':  '[source="53"]',
          'style':{
          'line-color':'#ff0000',
          'width':'0.1px',
          'opacity':'1',
          'target-arrow-color':'#ff0000'} 
        }]
    return default_stylesheet + new_styles


# TABS
@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value')) # issue: the tabs do not update whenever a new narrator is selected from the dropdown. perhaps another callback is needed.
def render_content(tab):
    if tab == 'bio':
        for b in bios:
            if b['NarratorID'] == currentNarrator:
                return html.Div([
                    b['Biography'] # issue: it doesn't read the \n characters
                ])
    elif tab == 'hadiths':
        for h in hadiths:
            if currentNarrator in h['NarrationChain']:
                return html.Div([
                    h['EnglishText'] # issue: it currently just returns after seeing the first hadith, fix so that it shows ALL of them - it must run the whole for loop before returning
                ])

## Run App

In [12]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
